<a href="https://colab.research.google.com/github/codeeye00/text-summarization/blob/master/Text_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
#working google drive ya RAB
#https://stackoverflow.com/questions/52385655/unable-to-locate-package-google-drive-ocamlfuse-suddenly-stopped-working


!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
--2019-11-24 18:23:08--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8003, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-11-24 18:23:10 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Please, open the following URL in a web browser: https://accounts

In [5]:
!pip install gensim
!pip install wget
  
import nltk
nltk.download('punkt')

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=6f376326d3480346331ceb1a2d30a0d2b803db8c09fdbe72c4ba5af3eff866d6
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
default_path = "drive/Colab Notebooks/Model 2/"#path for training text (article)
train_article_path = default_path + "sumdata/train/train.article.txt"  #path for training text output (headline)
train_title_path   = default_path + "sumdata/train/train.title.txt" #path for validation text (article)
valid_article_path = default_path + "sumdata/train/valid.article.filter.txt"#path for validation text output(headline)
valid_title_path   = default_path + "sumdata/train/valid.title.filter.txt"

In [0]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

def clean_str(sentence):
  sentence=re.sub("[#.]+","#",sentence)
  return sentence

def get_text_list(data_path,toy):
  with open(data_path,"r",encoding="utf-8")as f:
    if not tf:
      return [clean_str(x.strip()) for x in f.readlines()][:200000]
    else:
      return [clean_str(x.strip()) for x in f.readlines()][:50]

def build_dic(step,toy=False):
  if step=="train":
    train_article_list=get_test_list(train_article_path,toy)
    train_title_list=get_text_list(train_title_path,toy)

    word_counter=collections.Counter(words).most_common()
    word_dict=dict()
    word_dict["<padding>"]=0
    word_dict["<unk>"]=1
    word_dict["<s>"]=2
    word_dict["</s>"]=3
    for word, _ in word_counter:
      word_dict[word]=len(word_dict)
    with open(default_path + "word_dict.pickle","wb")as f:
      pickle.dump(word_dict,f)
  elif step=="valid":
    with open(default_path + "word_dict.pickle","rb")as f:
      word_dict=pickle.load(f)
  reversed_dict=dict(zip(word_dict.values(),word_dict.keys()))
  article_max_len=50
  summary_max_len=15
  return word_dict, reversed_dict, article_max_len,summary_max_len